In [13]:
from lightning.pytorch.cli import LightningArgumentParser, LightningCLI
import lightning.pytorch as pl
from lightning.pytorch.cli import OptimizerCallable, LRSchedulerCallable
from jsonargparse import lazy_instance
import torch

In [2]:
import typing

class LRSchedulerAdapterBase:
    def instantiate(self, parameters, batch_size):
        pass

class LRSchedulerTest(LRSchedulerAdapterBase):
    def __init__(self, lr_param:float) -> None:
        super().__init__()
        self.lr_param = lr_param
    def instantiate(self, parameters, batch_size):
        return self.lr_param


class TestModule(pl.LightningModule):
    def __init__(self, lrs:typing.Union[LRSchedulerCallable, LRSchedulerAdapterBase]=LRSchedulerTest(lr_param=8)) -> None:
        self.lrs = lrs
    
    def configure_optimizers(self):
        print(self.lrs.lr_param) 

In [3]:
LRSchedulerTest()

TypeError: LRSchedulerTest.__init__() missing 1 required positional argument: 'lr_param'

In [39]:
cli = LightningCLI(TestModule, # args=["--model.lrs=Adam"], 
        args=["--model.lrs=ReduceLROnPlateau", ],
        run=False)
cli.model.configure_optimizers()

/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/site-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['--ip=127.0.0.1', '--stdin=9003', '--control=9001', '--hb=9000', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"56e907ce-3087-42ac-8586-520bbf56a8be"', '--shell=9002', '--transport="tcp"', '--iopub=9004', '--f=/home/llr/cms/cuisset/.local/share/jupyter/runtime/kernel-v2-136478WDE6mus5Xpen.json'], args=['--model.lrs=ReduceLROnPlateau'].
  rank_zero_warn(


AttributeError: 'NoneType' object has no attribute 'get'

In [20]:
class TestModule2(pl.LightningModule):
    def __init__(self, opt:OptimizerCallable) -> None:
        self.opt = opt
    
    def configure_optimizers(self):
        print(self.opt([torch.nn.Parameter(torch.tensor(1.))])) 

In [23]:
cli = LightningCLI(TestModule2, # args=["--model.lrs=Adam"], 
        args=["--model.opt=AdamW", "--model.opt.lr=1e-5"],
        run=False)
cli.model.configure_optimizers()

/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/site-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['--ip=127.0.0.1', '--stdin=9049', '--control=9047', '--hb=9046', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"2d7413f8-9a5b-4780-9f77-ddb92488bf3b"', '--shell=9048', '--transport="tcp"', '--iopub=9050', '--f=/home/llr/cms/cuisset/.local/share/jupyter/runtime/kernel-v2-241813gzOio1uADbtH.json'], args=['--model.opt=AdamW', '--model.opt.lr=1e-5'].
  rank_zero_warn(
Global seed set to 863193537
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1e-05
    maximize: False
    weight_decay: 0.01
)


In [24]:
cli.model.configure_optimizers()

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1e-05
    maximize: False
    weight_decay: 0.01
)
